In [ ]:
import torch
import torch.nn as nn
import torchvision.datasets as dset
import torchvision.transforms as transforms 
from torch.utils.data import DataLoader

📌Q1. 가장 먼저 학습 데이터를 준비해보도록 하겠습니다. MNIST 데이터셋을 직접 Load해 봅시다. 데이터셋을 로드하고 DataLoader를 구현해보세요.

- DataLoader를 이용해 MNIST 데이터셋을 로드해봅시다.

In [ ]:
training_epochs = 15
batch_size = 100

root = './data'
mnist_train = dset.MNIST(root=root, train=True, transform=transforms.ToTensor(), download=True)
mnist_test = dset.MNIST(root=root, train=False, transform=transforms.ToTensor(), download=True)

train_loader = DataLoader(mnist_train, batch_size = batch_size, shuffle = True)
test_loader = DataLoader(mnist_test, batch_size = batch_size, shuffle = False)


📌Q2.데이터가 준비가 되었다면, 이제 그 데이터를 학습할 모델을 구현할 차례입니다. 그후 모델 안의 가중치를 초기화시켜보세요. 입력 데이터 형태에 맞도록 linear한 모델을 구성해보세요.

- MNIST 입력의 크기는 28x28입니다.

- 여기서 구현하는 linear 모델은 입력이 1차원이기 때문에 입력 차원을 맞춰보세요.


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
linear = torch.nn.Linear(784, 10, bias = True).to(device)
torch.nn.init.normal_(linear.weight)

Parameter containing:
tensor([[ 0.6558, -0.9865, -0.3268,  ...,  1.3842, -0.4537,  2.0292],
        [-0.4716,  0.4212,  0.8715,  ...,  0.6578,  0.1918, -0.0359],
        [ 0.8237, -0.7472, -0.8979,  ..., -0.2953,  0.3836,  0.6029],
        ...,
        [-0.8371,  0.8613,  0.5890,  ...,  0.3800,  0.6788, -0.5509],
        [-0.5659, -0.3064, -0.8841,  ..., -0.2912,  0.2062, -1.0937],
        [ 0.6580,  0.0699,  1.1987,  ...,  1.0337, -0.7570, -0.0434]],
       requires_grad=True)

📌Q3. 위에서 구현한 모델을 학습시키기 위해서는 loss 함수와 opitmizer가 필요합니다.
아래 제시된 loss 함수와 optimizer를 구현해보세요. Loss 함수와 optimizer는 모델 안
의 가중치를 업데이트 할 때 사용됩니다.

- 옵티마이저는 SGD, Loss는 Cross Entropy Loss를 사용합니다.

In [ ]:
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.SGD(linear.parameters(), lr=0.1)

📌Q4. 3번 문제까지 해결하셨다면, 이제 학습을 위한 준비는 거의 끝났다고 볼 수 있습니다.
위 구현 함수들을 이용해 학습 Loop를 구현해보세요.

- 위에서 구현한 모델, optimzer, loss fn 등을 이용해 학습을 구현해주세요.

In [ ]:
for epoch in range(training_epochs):
    for i, (imgs, labels) in enumerate(train_loader):
        imgs, labels = imgs.to(device), labels.to(device)        
        imgs = imgs.view(-1, 28*28)

        outputs = linear(imgs)
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        _,argmax = torch.max(outputs, 1)
        accuracy = (labels == argmax.to(device)).float().mean()
        
        if (i+1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {: .4f}, Accuracy: {: .2f}%'.format(
                epoch+1, training_epochs, i+1, len(train_loader), loss.item(), accuracy.item()* 100))


📌Q5. 학습이 완료되면, 모델이 잘 동작하는지 테스트가 필요합니다. 데이터로드 파트에서
준비했던 테스트 데이터를 이용해 테스트를 진행해봅시다. 아래 테스트 코드를 완성해보
세요.

In [ ]:
linear.eval()

with torch.no_grad():
    correct = 0
    total = 0
    for i, (imgs, labels) in enumerate(test_loader):
        imgs, labels = imgs.to(device), labels.to(device)
        imgs = imgs.view(-1, 28 * 28)

        outputs = linear(imgs)

        _, argmax = torch.max(outputs, 1) 
        total += imgs.size(0)
        correct += (labels == argmax).sum().item()
    
    print('Test accuracy for {} images: {: .2f}%'.format(total, correct / total * 100))